In [3]:
#importing essential libraries for pyspark and sql
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *
import mysql.connector
import pandas as pd

#create spark session
spark = SparkSession.builder.config("spark.jars", "/home/hduser/mysql-connector-java-5.1.47/mysql-connector-java-5.1.47.jar") \
    .master("local").appName("ETL to SQL").getOrCreate()
spark

In [8]:
#connecting to localhost sql database
 
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database ='etl'
)
 
# Creating an instance of 'cursor' class
# which is used to execute the 'SQL'
# statements in 'Python'
cursor = mydb.cursor()

In [5]:
def sql_upload(df,table):
    pd_dataframe=df.toPandas()
    cursor.execute("CREATE IF NOT EXISTS "+table+" ( `id` INT NOT NULL AUTO_INCREMENT , `date` DATETIME NOT NULL , `high` DOUBLE NOT NULL , `low` DOUBLE NOT NULL , `close` DOUBLE NOT NULL , PRIMARY KEY (`id`))")
    for index, row in pd_dataframe.iterrows():
        cursor.execute("INSERT INTO "+table+" (date,high,low,close) values(%s,%s,%s,%s)",(row['date'],row['high'],row['low'],row['close']))
        mydb.commit()
    print("Successfully uploaded to sql table")

In [7]:
btc_df = spark.read.csv('csv\\eth.csv', header=True, sep=",").cache()

In [41]:
sql_upload(btc_df,"ethereum1")
print("test")

Successfully uploaded to sql table
test


In [30]:
eth_df = spark.read.csv('csv\\ethereum.csv', header=True, sep=",").cache()
ada_df = spark.read.csv('csv\\cardano.csv', header=True, sep=",").cache()
polka_df = spark.read.csv('csv\\polkadot.csv', header=True, sep=",").cache()

In [31]:
sql_upload(eth_df,"ethereum")
sql_upload(ada_df,"cardano")
sql_upload(polka_df,"polkadot")

Successfully uploaded to sql table
Successfully uploaded to sql table
Successfully uploaded to sql table
